# DX트랙 5차 미니프로젝트 
## 중증질환 예측 모델링

 * Target : 중증질환 (뇌경색, 뇌출혈, 복부손상, 심근경색)
 * 데이터 분석 결과를 바탕으로 Target에 영향을 주는 Feature 전처리 (함수 정의)
 * 머신러닝/딥러닝 모델링 후 성능 비교
 * 최적AI 모델 선정 및 저장
 * 새로운 출동 이력에 제시된 환자의 증상을 바탕으로 중증 질환 예측 함수 정의

In [1]:
## windows 도스창 기준입니다.
## keras 버전 확인
!pip list | find "keras" 

keras                         2.11.0


In [2]:
# keras 버전 맞추기
!pip install keras==2.11.0 --upgrade --user

In [33]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler

plt.rcParams['font.family'] = 'Malgun Gothic'

In [19]:
# 응급 출동 데이터 불러오기
# 파일명 : 119_emergency_dispatch.csv
# 중증 질환이 ['심근경색', '복부손상', '뇌경색', '뇌출혈']인 데이터만 추출
# 데이터 랜덤으로 섞기

data = pd.read_csv('119_emergency_dispatch_1.csv', encoding='cp949')
desease = data[data['중증질환'].isin(['심근경색', '복부손상', '뇌경색', '뇌출혈'])]

# 데이터 랜덤으로 섞기

desease = desease.sample(frac=1).reset_index(drop=True)


In [20]:
desease.head()


,ID,출동일시,이름,성별,나이,체온,수축기 혈압,이완기 혈압,호흡 곤란,간헐성 경련,...,통증,만지면 아프다,무감각,마비,현기증,졸도,말이 어눌해졌다,시력이 흐려짐,중증질환,이송 시간
0,P16152,2023-03-07,유**,여성,74,36.5,95,75,1,0,...,0,1,0,0,0,0,0,0,뇌경색,48
1,P20047,2023-03-21,태**,남성,54,36.4,107,75,0,0,...,0,0,0,0,0,0,0,0,심근경색,52
2,P15103,2023-01-09,마**,남성,72,36.9,116,67,1,0,...,0,1,1,0,0,0,0,0,뇌경색,8
3,P22952,2023-02-18,팽**,남성,23,36.2,129,83,1,0,...,1,0,0,0,0,1,0,0,심근경색,66
4,P19589,2023-04-01,반**,여성,57,36.2,104,80,0,0,...,0,0,1,0,0,0,0,0,뇌경색,51


## 1) 학습용, 평가용 데이터 준비하기

 * 함수로 전처리 과정을 정의
   * Target : 중증질환
   * 전처리 대상 : 발열, 고혈압, 저혈압
   * Feature : '체온', '수축기 혈압', '이완기 혈압', '호흡 곤란', '간헐성 경련', '설사', '기침', '출혈', '통증', '만지면 아프다', 
           '무감각', '마비', '현기증', '졸도', '말이 어눌해졌다', '시력이 흐려짐', '발열', '고혈압', '저혈압'
   * Feature와 Target을 분리하여 결과값 return
 

In [21]:
def preprocessing(desease):
    desease['발열'] = [1 if i >=37 else 0 for i in desease['체온']]
    desease['고혈압'] = [1 if i >=140 else 0 for i in desease['수축기 혈압']]
    desease['저혈압'] = [1 if i<=90 else 0 for i in desease['수축기 혈압']]
    
    target='중증질환'
    X=desease[['체온', '수축기 혈압', '이완기 혈압', '호흡 곤란', '간헐성 경련', '설사', '기침', '출혈', '통증', '만지면 아프다', '무감각', '마비', '현기증', '졸도', '말이 어눌해졌다', '시력이 흐려짐', '발열', '고혈압', '저혈압']]
    Y=desease.loc[:,target]
    # xgboost나 딥러닝의 경우 라벨링이 무조건 필요하므로 라벨 전환
    Y=Y.map({'뇌출혈':0,'심근경색':1,'복부손상':2,'뇌경색':3})
    return X, Y

In [22]:
# x, y 데이터 나누기
# preprocessing 함수 활용하여 전처리까지 완료하고 데이터 분리

x, y = preprocessing(desease)

In [23]:
desease['중증질환'].unique()

array(['뇌경색', '심근경색', '뇌출혈', '복부손상'], dtype=object)

In [24]:
# 데이터 분할 비율: 학습데이터 7 : 검증데이터 3
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)


## 2) 모델링

 * 활용 모델 : DecisionTreeClassifier, RandomForestClassifier, XGBClassifier, DNN
 * 성능 평가 : accuracy_score

In [25]:
## Decision Tree
## 1) 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

## 2) 선언하기
model_dt=DecisionTreeClassifier()

## 3) 학습하기
model_dt.fit(x_train,y_train)

## 4) 예측하기
y_pred=model_dt.predict(x_test)
## 5) 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1088   11   29   65]
 [   8 1210   27    9]
 [  33    7 1222  147]
 [  60   13  135 1044]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1193
           1       0.98      0.96      0.97      1254
           2       0.86      0.87      0.87      1409
           3       0.83      0.83      0.83      1252

    accuracy                           0.89      5108
   macro avg       0.90      0.89      0.89      5108
weighted avg       0.89      0.89      0.89      5108



In [92]:
## RandomForest
## 1) 불러오기
from sklearn.ensemble import RandomForestClassifier

## 2) 선언하기
model_rf=RandomForestClassifier(max_depth=20)

## 3) 학습하기
model_rf.fit(x_train,y_train)

## 4) 예측하기
y_pred=model_rf.predict(x_test)

## 5) 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1106    5   14   68]
 [   1 1235   13    5]
 [   9    6 1302   92]
 [  27   12  131 1082]]
              precision    recall  f1-score   support

           0       0.97      0.93      0.95      1193
           1       0.98      0.98      0.98      1254
           2       0.89      0.92      0.91      1409
           3       0.87      0.86      0.87      1252

    accuracy                           0.93      5108
   macro avg       0.93      0.93      0.93      5108
weighted avg       0.93      0.93      0.93      5108



In [32]:
## XGBoost
## 1) 불러오기
from xgboost import XGBClassifier

## 2) 선언하기
model_xgb=XGBClassifier()

## 3) 학습하기
model_xgb.fit(x_train,y_train)


## 4) 예측하기
y_pred=model_xgb.predict(x_test)

## 5) 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1116    5   10   62]
 [   2 1233   14    5]
 [  17   10 1294   88]
 [  32   13  132 1075]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1193
           1       0.98      0.98      0.98      1254
           2       0.89      0.92      0.91      1409
           3       0.87      0.86      0.87      1252

    accuracy                           0.92      5108
   macro avg       0.93      0.92      0.92      5108
weighted avg       0.92      0.92      0.92      5108



---

In [74]:
# 딥러닝 이전에 MinMaxScaling 진행
x, y = preprocessing(desease)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

In [75]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [76]:
x_train.shape

(11916, 19)

In [77]:
## DNN
## 1) 불러오기

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.backend import clear_session
from keras.callbacks import EarlyStopping

# 메모리 정리
clear_session()

## 2) 선언하기
n_features=x_train.shape[1]

model=Sequential([Dense(4,input_shape=(n_features,),activation='softmax')])

# ## 3) 학습하기
callback=EarlyStopping(monitor='loss',patience=3)

model.compile(optimizer=Adam(learning_rate=0.05), loss='sparse_categorical_crossentropy')
history=model.fit(x_train,y_train, epochs=50, validation_split=0.2, callbacks=[callback]).history              

Epoch 1/50
298/298 [==============================] - 2s 5ms/step - loss: 0.4265 - val_loss: 0.3780
Epoch 2/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3453 - val_loss: 0.3689
Epoch 3/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3424 - val_loss: 0.3752
Epoch 4/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3394 - val_loss: 0.3711
Epoch 5/50
298/298 [==============================] - 1s 4ms/step - loss: 0.3393 - val_loss: 0.3773
Epoch 6/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3413 - val_loss: 0.3832
Epoch 7/50
298/298 [==============================] - 1s 4ms/step - loss: 0.3382 - val_loss: 0.3788
Epoch 8/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3410 - val_loss: 0.3786
Epoch 9/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3385 - val_loss: 0.3720
Epoch 10/50
298/298 [==============================] - 1s 3ms/step - loss: 0.3410 - val_loss: 0.3719

In [78]:
# ## 4) 예측하기
                  
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

# ## 5) 평가하기
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))  

160/160 [==============================] - 0s 2ms/step
[[1068    5   26   94]
 [   4 1217   27    6]
 [   8   29 1261  111]
 [  45   16  148 1043]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.92      1193
           1       0.96      0.97      0.97      1254
           2       0.86      0.89      0.88      1409
           3       0.83      0.83      0.83      1252

    accuracy                           0.90      5108
   macro avg       0.90      0.90      0.90      5108
weighted avg       0.90      0.90      0.90      5108



## 3) 최적 모델 선정 및 저장

#### 최적의 모델
##### 정확도 기준
- Decision Tree: 82%
- RandomForest: 93%
- XGBoost: 92%
- DNN: 90%

In [93]:
## 모델 저장하기
from joblib import dump

dump(model_rf, 'model_rf.joblib')

['model_rf.joblib']

## 4) 새로운 증상으로 중증질환 예측하기

In [95]:
model_rf

RandomForestClassifier(max_depth=20)

In [96]:
new_dispatch = {
    "ID" : [500001],
    "출동일시" :['2023-04-18'],
    "이름" : ['최**'],
    "성별" : ["여성"],
    "나이" : [35],
    "체온" : [36.5],
    "수축기 혈압" : [125],
    "이완기 혈압" : [85],
    "호흡 곤란":[0],
    "간헐성 경련":[0],
    "설사":[0],
    "기침":[0],
    "출혈":[1],
    "통증":[0],
    "만지면 아프다":[0],
    "무감각":[1],
    "마비":[1],
    "현기증":[0],
    "졸도":[1],
    "말이 어눌해졌다":[0],
    "시력이 흐려짐":[0],
    "중증질환" : [""],
    "이송 시간" : [0]
}

new_data = pd.DataFrame(new_dispatch)

In [97]:
new_data

,ID,출동일시,이름,성별,나이,체온,수축기 혈압,이완기 혈압,호흡 곤란,간헐성 경련,...,통증,만지면 아프다,무감각,마비,현기증,졸도,말이 어눌해졌다,시력이 흐려짐,중증질환,이송 시간
0,500001,2023-04-18,최**,여성,35,36.5,125,85,0,0,...,0,0,1,1,0,1,0,0,,0


In [100]:
# 전처리 과정 진행하기 preprocessing 함수 활용

new_x, new_y = preprocessing(new_data)

In [99]:
# 모델 불러오기

from joblib import load

model_rf=load('model_rf.joblib')

In [127]:
# 중증질환 예측하기
y_pred=model_rf.predict(new_x)


# 중증질환 명칭으로 표시하기

sym_list = ['뇌경색', '뇌출혈', '복부손상', '심근경색']
if y_pred==0:
    print('model_rf 결과: 뇌경색')
elif y_pred==1:
    print('model_rf 결과: 뇌출혈')
elif y_pred==2:
    print('model_rf 결과: 복부손상')
else:
    print('model_rf 결과: 심근경색')

model_rf 결과: 뇌경색


C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [125]:
# 예측 함수 정의하기
# input : 출동 이력 데이터 (new_dispatch, 딕셔너리 형태임)
# output : 중증 질환 명칭
# 함수 안에는 출동 이력 데이터를 전처리하는 과정이 필요함

def predict_disease(new_dispatch):
    
    sym_list = ['뇌경색', '뇌출혈', '복부손상', '심근경색']
    
    new_data = pd.DataFrame(new_dispatch)
    new_x, new_y = preprocessing(new_data)
    
    from joblib import load
    model_m = load('model_rf.joblib')
    
    y_pred = model_m.predict(new_x)
    
    return sym_list[int(y_pred)]

In [126]:
# new_dispatch dictionary 형태로 전달됨

predict_disease(new_dispatch)

C:\Users\User\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


'뇌경색'